In [1]:
from pathlib import Path

In [2]:
import re

In [3]:
!head -150 IDMS_RECORDS_cpy.txt | tail -50

0          1        5,473        3,664        1,809            0            0                                                        
           2        5,473        3,664        1,809            0            0                                                        
           3        5,473        3,664        1,809            0            0                                                        
           4        5,473        3,664        1,809            0            0                                                        
           5        5,473        3,664        1,809            0            0                                                        
           6        5,473        3,664        1,809            0            0                                                        
           7        5,473        3,664        1,809            0            0                                                        
           8        5,473        3,664        1,809           

# Initial cleaning

In [4]:
# !head -50 IDMS_RECORDS_cpy.txt

# p = Path( "2022-10-14_Whitespace_stripped.txt" )

# raw_text = p.read_text(encoding='latin1')

# len( raw_text )

# print( raw_text[:1000] )

# 

# p = re.compile( r'^1REPORT.*?^0$', flags = re.MULTILINE | re.DOTALL )

# len( p.findall( raw_text ) )

# removed_headers = p.sub( "", raw_text )

# len( removed_headers )

In [5]:
out = Path( "2022-10-14_Stripped_IDMS_RECORDS.txt" )

In [6]:
#out.write_text( removed_headers )

In [7]:
# reload after some hand-editing
removed_headers = out.read_text()

In [8]:
len( removed_headers )

15348439

In [9]:
lines = removed_headers.split( '\n' )

In [10]:
len( lines )

171643

In [11]:
p = re.compile( r'^\s+' )

In [12]:
p.findall( lines[0] )

[' ']

In [13]:
from collections import Counter

In [14]:
[ i for i, _ in enumerate( lines ) if len( p.findall( _ ) ) == 0 ]

[171642]

In [15]:
lines = [ _ for _ in lines if _ != "" ]

In [16]:
len( lines )

171642

In [17]:
[ i for i, _ in enumerate( lines ) if len( p.findall( _ ) ) == 0 ]

[]

In [18]:
leading_space_count = [ len( p.findall( _ )[0] ) for _ in lines if len( p.findall( _ ) ) > 0 ]

In [19]:
Counter( leading_space_count )

Counter({1: 3661,
         5: 18569,
         12: 31763,
         14: 2902,
         9: 31042,
         26: 13934,
         11: 62916,
         13: 63,
         10: 6756,
         27: 3,
         22: 33})

In [20]:
table_index = -1
table_indices = [0]*len( lines )
for i in range( len( lines ) ):
    if lines[i][1] != ' ':
        table_index += 1
    table_indices[ i ] = table_index

In [21]:
#table_indices

In [22]:
len( leading_space_count ) == len( lines )

True

In [23]:
import pandas as pd

In [24]:
df = pd.DataFrame( dict(
    leading_space_count = leading_space_count,
    table_index = table_indices,
    lines = lines ))

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171642 entries, 0 to 171641
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   leading_space_count  171642 non-null  int64 
 1   table_index          171642 non-null  int64 
 2   lines                171642 non-null  object
dtypes: int64(2), object(1)
memory usage: 3.9+ MB


In [26]:
df[ df['leading_space_count'] == 1 ]

,leading_space_count,table_index,lines
0,1,0,ABSDOMN VER 1 ...
11,1,1,ACCRLINK VER 1 ...
104,1,2,ACCRLINK VER 2 ...
168,1,3,ACCT VER 1 ...
184,1,4,ACCTG-CNTL VER 1 ...
...,...,...,...
171288,1,3656,502-REPT-DTL-RCRD VER 1 ...
171366,1,3657,502-REPT-DTL-RCRD VER 2 ...
171433,1,3658,502-REPT-DTL-RCRD VER 4 ...
171505,1,3659,502-TEMP-DTL-RCRD VER 1 ...


In [27]:
text_clumps = df.groupby( 'leading_space_count' ).apply( lambda group: group['lines'].to_list() )

In [28]:
text_clumps.info()

<class 'pandas.core.series.Series'>
Int64Index: 11 entries, 1 to 27
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
11 non-null     object
dtypes: object(1)
memory usage: 176.0+ bytes


In [29]:
[ len( _ ) for _ in text_clumps ]

[3661, 18569, 31042, 6756, 62916, 31763, 63, 2902, 33, 13934, 3]

# TABLE NAMES

In [30]:
text_clumps[1][:20]

[' ABSDOMN                          VER     1    32      D                                    10/15/87 09/29/87',
 ' ACCRLINK                         VER     1   176      D                                    03/09/98 02/21/84',
 ' ACCRLINK                         VER     2   176      D                                    05/15/98 05/15/98',
 ' ACCT                             VER     1    40      S                                    09/24/87 09/21/87',
 ' ACCTG-CNTL                       VER     1   100      D                                    02/07/89 05/26/87',
 ' ACH-RCRD                         VER     1   148      D                                             06/22/84',
 ' ACQD-PROP-MAP-RCRD               VER     1   141      C                                    07/31/87 06/30/87',
 ' ACQD-PROP-QUEUE-RCRD             VER     1   260      C                                    01/20/93 06/30/87',
 ' ACQD-PROP-SALES-RCRD             VER     1   286      D                              

In [31]:
Counter( [ len(_) for _ in text_clumps[1] ] )

Counter({109: 3661})

In [32]:
p = re.compile( r'\s(\S+)\s+VER\s+(\d+)' )

In [33]:
table_info_list = [ p.match( _ ).groups(1) for _ in text_clumps[1] ]

In [34]:
len( table_info_list )

3661

In [35]:
table_info_list[:10]

[('ABSDOMN', '1'),
 ('ACCRLINK', '1'),
 ('ACCRLINK', '2'),
 ('ACCT', '1'),
 ('ACCTG-CNTL', '1'),
 ('ACH-RCRD', '1'),
 ('ACQD-PROP-MAP-RCRD', '1'),
 ('ACQD-PROP-QUEUE-RCRD', '1'),
 ('ACQD-PROP-SALES-RCRD', '1'),
 ('ACQD-PROP-SOLD', '1')]

In [36]:
table_names, table_vers = zip(*table_info_list )

In [37]:
created_dates = pd.to_datetime( [ _[101:109] for _ in text_clumps[1] ] )

In [38]:
len( created_dates )

3661

In [39]:
updated_dates = pd.to_datetime( [ _[92:100] for _ in text_clumps[1] ], errors='coerce' )

In [40]:
updated_dates

DatetimeIndex(['1987-10-15', '1998-03-09', '1998-05-15', '1987-09-24',
               '1989-02-07',        'NaT', '1987-07-31', '1993-01-20',
               '1994-09-19', '1991-02-22',
               ...
               '2000-01-28', '1992-06-03', '1989-11-15', '1994-09-16',
               '1983-09-15', '2002-12-09',        'NaT', '2002-03-14',
               '2002-12-09', '2002-03-14'],
              dtype='datetime64[ns]', length=3661, freq=None)

In [41]:
table_info = pd.DataFrame( data=dict(
    table_names=table_names,
    table_vers=table_vers,
    created_dates=created_dates,
    updated_dates= updated_dates
))

In [42]:
table_info.head(20)

,table_names,table_vers,created_dates,updated_dates
0,ABSDOMN,1,1987-09-29,1987-10-15
1,ACCRLINK,1,1984-02-21,1998-03-09
2,ACCRLINK,2,1998-05-15,1998-05-15
3,ACCT,1,1987-09-21,1987-09-24
4,ACCTG-CNTL,1,1987-05-26,1989-02-07
5,ACH-RCRD,1,1984-06-22,NaT
6,ACQD-PROP-MAP-RCRD,1,1987-06-30,1987-07-31
7,ACQD-PROP-QUEUE-RCRD,1,1987-06-30,1993-01-20
8,ACQD-PROP-SALES-RCRD,1,1994-09-01,1994-09-19
9,ACQD-PROP-SOLD,1,1989-04-26,1991-02-22


# 5's are for metadata

In [43]:
text_clumps[5][:20]

['     PREPARED BY          AWMC11',
 '     REVISED BY           AWMC11',
 '     DESCRIPTION          USER DOMAIN',
 '     RECORD               ABSDOMN                                  VER     1 D  SUFFIX -4541',
 '     PREPARED BY          AA-KGM',
 '     REVISED BY           ASL034',
 '     DESCRIPTION          ACCRUAL LINK RECORD',
 '     RFA-ID               H-3973',
 '     RFA-ID               H-3581A',
 '     DEFINITION',
 '     RECORD               ACCRLINK                                 VER     1 D  PREFIX W019-',
 '     PREPARED BY          ASL70M',
 '     REVISED BY           ASL70M',
 '     DESCRIPTION          ACCRUAL LINK RECORD',
 '     RFA-ID               H-3973',
 '     RFA-ID               H-3581A',
 '     SAME AS              ACCRLINK                                 VER     1',
 '     DEFINITION',
 '     RECORD               ACCRLINK                                 VER     2 D  PREFIX W019-',
 '     PREPARED BY          ASL052']

In [44]:
p = re.compile( r'     (\S+)' )

In [45]:
Counter( [ p.match( _ ).groups(1)[0] for _ in text_clumps[5] ] )

Counter({'PREPARED': 3629,
         'REVISED': 3032,
         'DESCRIPTION': 3176,
         'RECORD': 4126,
         'RFA-ID': 729,
         'DEFINITION': 1166,
         'SAME': 573,
         'IN': 1737,
         'COMMENT': 203,
         'PURPOSE': 63,
         'PRIMARY': 27,
         'FMHA-TRANS': 19,
         'LANGUAGE': 44,
         'SUB-SYSTEM': 2,
         'MODE': 39,
         'EGEN': 1,
         'USER': 1,
         'MEDIUM': 1,
         'FREQUENCY': 1})

In [46]:
p = re.compile( r'^ (\S+)\s+VER\s+(\d+).*?DESCRIPTION\s+(.*?)$', flags=re.DOTALL | re.MULTILINE ) 

In [47]:
desc_table_names, desc_table_vers, desc_descs = zip( *p.findall( removed_headers ) )

In [48]:
len( desc_table_names )

3184

In [49]:
len( desc_table_vers )

3184

In [50]:
len( desc_table_names) 

3184

In [51]:
descs = pd.DataFrame( dict(
    table_names = desc_table_names,
    table_vers = desc_table_vers,
    descriptions = desc_descs ))

In [52]:
table_info.shape

(3661, 4)

In [53]:
descs.shape

(3184, 3)

In [54]:
merged_table_info = table_info.merge( descs, on=['table_names', "table_vers"], how='left' )

In [55]:
merged_table_info

,table_names,table_vers,created_dates,updated_dates,descriptions
0,ABSDOMN,1,1987-09-29,1987-10-15,USER DOMAIN
1,ACCRLINK,1,1984-02-21,1998-03-09,ACCRUAL LINK RECORD
2,ACCRLINK,2,1998-05-15,1998-05-15,ACCRUAL LINK RECORD
3,ACCT,1,1987-09-21,1987-09-24,DFIS ACCOUNT
4,ACCTG-CNTL,1,1987-05-26,1989-02-07,ACCOUNTING MONTH ACTIVITY TOTALS
...,...,...,...,...,...
3656,502-REPT-DTL-RCRD,1,1984-06-28,2002-12-09,COPY MODULE FOR 502-REPORT
3657,502-REPT-DTL-RCRD,2,2003-06-13,NaT,COPY MODULE FOR 502-REPORT
3658,502-REPT-DTL-RCRD,4,2002-03-12,2002-03-14,COPY MODULE FOR 502-REPORT
3659,502-TEMP-DTL-RCRD,1,1989-01-30,2002-12-09,WORK RECORD FOR 502 REPORT


# 9's indicate "COPIED IN PROGRAM", "FILE", AND "SUBSCHEMA"

In [56]:
#text_clumps[9][-200:]
text_clumps[9][-20:]

['         SUBSCHEMA            SSPMF03M  OF SCHEMA SCMMF01M             VER     1',
 '         SUBSCHEMA            SSPMF03P  OF SCHEMA SCMMF01P             VER     1',
 '         SUBSCHEMA            XSSPMF03  OF SCHEMA XSCMMF01             VER     1',
 '         COPIED IN PROGRAM    SBMDD010                                 VER     1',
 '         COPIED IN PROGRAM    SCMDD021                                 VER     1',
 '         COPIED IN PROGRAM    SBDLD142                                 VER     1',
 '         COPIED IN PROGRAM    SDDLD355                                 VER     5',
 '         COPIED IN PROGRAM    SDDLD355                                 VER     3',
 '         COPIED IN PROGRAM    SBDLD101                                 VER     6',
 '         COPIED IN PROGRAM    SDDLD355                                 VER     1',
 '         COPIED IN PROGRAM    SBDLD101                                 VER     4',
 '         COPIED IN PROGRAM    SDDLD355                         

# 10's through 12's are data elements

In [57]:
text_clumps[10][-10:]

['          14500   05 DTE-LST-CNCLTN-RDA                       9(6)                             DISPLAY',
 '          14600   05 PRIN-INSTLMT-BOND-ACCTS-RDA              S9(7)V99                         COMP-3',
 '          14700   05 INSTLMT-OC-DATA-320-RDA                                                   DISPLAY',
 '          14800    10 DTE-INSTLMT-DUE-320-RDA                 9(6)                             DISPLAY',
 '          14900    10 INSTLMT-DUE-AMT-320-RDA                 S9(06)V99                        COMP-3',
 '          15000    10 INSTLMT-TYP-CDE-320-RDA                 9(1)                             DISPLAY',
 '          15100    10 MTURD-CDE-DIR-RE-320-RDA                9(1)                             DISPLAY',
 '          15200   05 RCVB-AMT-DUE-RDA                         S9(09)V99                        COMP-3',
 '          15300   05 RCVB-AMT-PD-RDA                          S9(09)V99                        COMP-3',
 '          15400   05 FILLER            

In [58]:
text_clumps[11][-10:]

['           4600   05 W502-RGSTR-TYP-LST-CHNG                  9(2)                             DISPLAY',
 '           4700   05 FILLER                                   X(0014)                          DISPLAY',
 '           4800   05 W502-DTE-APPRVL-OBLGN                    9(5)                             DISPLAY',
 '           4900   05 W502-ETHNCTY-TYP-CDE                     X                                DISPLAY',
 '           5000   05 W502-RACE-TYP-CDE                        X                                DISPLAY',
 '           5100   05 W502-RACE-DTRMNTN-CDE                    X                                DISPLAY',
 '           5200   05 W502-ST-CDE                              9(2)                             DISPLAY',
 '           5300   05 FILLER                                   X(0002)                          DISPLAY',
 '           5400   05 W502-CR-REPT-FEE-CDE                     9(1)                             DISPLAY',
 '           5500   05 W502-PROJ-NBR-

In [59]:
text_clumps[12][-10:]

['            900    10 W502-PPB-CDE-1ST                        9                                DISPLAY',
 '            100   05 W502-RCRD-TYP                            9(2)                             DISPLAY',
 '            200   05 W502-RGSTR-TYP-CDE                       9(2)                             DISPLAY',
 '            300   05 W502-MO-OBLGN                            9(2)                             DISPLAY',
 '            400   05 W502-FY                                  9(2)                             DISPLAY',
 '            500   05 W502-APROPTN-MAJ-CLS-CDE                                                  DISPLAY',
 '            600    10 W502-APROPTN-CDE                        9(2)                             DISPLAY',
 '            700    10 W502-MAJ-CLS                            9(2)                             DISPLAY',
 '            800   05 W502-PROG-PLANG-BDGTG-CDE                                                 DISPLAY',
 '            900    10 W502-PPB-CDE-

In [60]:
all_data_elements = [ *text_clumps[10], *text_clumps[11], *text_clumps[12] ]

In [61]:
len( all_data_elements )

101435

# 13's are data elements as well as descriptions

In [62]:
len(text_clumps[13])

63

In [63]:
text_clumps[13][:20]

['             10   05 TRNSCTN-CDE-UPDTE                        X(2)                             DISPLAY',
 '             20   05 CASE-NBR                                 X(15)                            DISPLAY',
 '             30   05 OK-TO-PROC-CDE                           X(1)                             DISPLAY',
 '             40   05 FD-CDE-2                                 X(2)                             DISPLAY',
 '             50   05 LN-TYP-ABBR                              X(2)                             DISPLAY',
 '             60   05 LN-NBR                                   X(2)                             DISPLAY',
 '             70   05 COLLCTN-TYP-CDE-MISC                     X(2)                             DISPLAY',
 '             80   05 FILLER                                   X(0005)                          DISPLAY',
 '             90   05 PYMT-AMT                                 X(7)                             DISPLAY',
 '             1000 INT-RATE-CTGRY IS

In [64]:
data_elements_13 = [ _ for _ in text_clumps[13] if _[16] == ' ' ]

In [65]:
len( data_elements_13 )

9

In [66]:
data_elements_13

['             10   05 TRNSCTN-CDE-UPDTE                        X(2)                             DISPLAY',
 '             20   05 CASE-NBR                                 X(15)                            DISPLAY',
 '             30   05 OK-TO-PROC-CDE                           X(1)                             DISPLAY',
 '             40   05 FD-CDE-2                                 X(2)                             DISPLAY',
 '             50   05 LN-TYP-ABBR                              X(2)                             DISPLAY',
 '             60   05 LN-NBR                                   X(2)                             DISPLAY',
 '             70   05 COLLCTN-TYP-CDE-MISC                     X(2)                             DISPLAY',
 '             80   05 FILLER                                   X(0005)                          DISPLAY',
 '             90   05 PYMT-AMT                                 X(7)                             DISPLAY']

In [67]:
all_data_elements.extend( data_elements_13 )

In [68]:
len( all_data_elements )

101444

In [69]:
p = re.compile( r'^\s+(\d+)\s+(\d+) (\S+)' )#\s+(\S+)?\s+(\S)$' )
#p = re.compile( r'^\s+(\d+)\s+(\d+) (\S+)\s+(\S+)?\s+(\S)$' )

In [70]:
p.match( all_data_elements[0] ).groups()

('10000', '05', 'AW-U-YR')

In [71]:
data_steps, indent, element_name = zip( *[ p.match( _ ).groups() for _ in all_data_elements ] )

In [72]:
len( data_steps)

101444

In [73]:
'          10000   05 AW-U-YR                                  99                               DISPLAY'.index( 'DISPLAY' )

95

In [74]:
p = re.compile( r'(\S+)' )

In [75]:
#pd.DataFrame(  ).to_csv( '2022-10-17_IDMS_COBOL_datatypes.csv ' )

In [76]:
unique_data_types = set( [ p.match( _[62:95] ).groups()[0] for _ in all_data_elements if p.match( _[62:95] ) ] )

In [77]:
unique_data_types = sorted( list( unique_data_types ), key=lambda t: ( len(t), t ) )

In [78]:
Path( '2022-10-17_IDMS_COBOL_datatypes.txt' ).write_text( '\n'.join( unique_data_types ) )

5115

# 14's are DEFINITION and COMMENTs only

In [79]:
text_clumps[14][:20]

['              100 THIS RECORD IS USED TO PASS TO AND',
 '              200 FROM INFORMATION REQUIRED TO COMPLETE',
 '              300 AN ACCRUAL',
 '              100 THIS RECORD IS USED TO PASS TO AND',
 '              200 FROM INFORMATION REQUIRED TO COMPLETE',
 '              300 AN ACCRUAL',
 '              100 THE AMOUNT IN THAT DFIS ACCOUNT.',
 '              100 AT THE END OF EACH ACCOUNTING MONTH BEFORE THE',
 '              200 STATE-CNTL AND THE CYCLE-CNTL RECORDS ARE RESET',
 '              300 TO ZERO FOR THE NEXT MONTH, THE ACTIVITY CONTROL',
 '              400 TOTALS FOR THE COMPLETED MONTH WILL BE MOVED',
 '              500 INTO THIS 15 MONTH ACCTG-CNTL FILE.',
 '              100 ACQUIRED PROPERTY QUEUE RECORD',
 '              100 STANDARD RECORD DESCRIPTION FOR ACQUIRED',
 '              200 PROPERTY TAPE FILE',
 '              100 PICTURE AND USAGE FOR SOME ELEMENTS HAVE BEEN OVERRIDDEN',
 '              200 TO CREATE RECORD AS ESTABLISHED DURING CONVERSION',
 '

In [80]:
comment_lines = df[ df['leading_space_count'] == 14 ]

In [81]:
comments = comment_lines.groupby( 'table_index' ).apply( lambda group: " ".join( _[18:] for _ in group['lines'].values  ) )

In [82]:
comments[2]

'THIS RECORD IS USED TO PASS TO AND FROM INFORMATION REQUIRED TO COMPLETE AN ACCRUAL'

In [83]:
comments

table_index
1       THIS RECORD IS USED TO PASS TO AND FROM INFORM...
2       THIS RECORD IS USED TO PASS TO AND FROM INFORM...
3                        THE AMOUNT IN THAT DFIS ACCOUNT.
4       AT THE END OF EACH ACCOUNTING MONTH BEFORE THE...
7                          ACQUIRED PROPERTY QUEUE RECORD
                              ...                        
3656    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3657    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3658    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3659    THIS RECORD IS USED FOR A WORKING STORAGE AREA...
3660    THIS RECORD IS USED FOR A WORKING STORAGE AREA...
Length: 1307, dtype: object

In [84]:
comments.name = 'comments'

In [85]:
merged_table_info = merged_table_info.merge( comments, how='left', left_index=True, right_index=True )

In [86]:
# Do not sort!
merged_table_info = merged_table_info.sort_values( ['created_dates', 'table_names', 'table_vers'] )

In [87]:
merged_table_info.to_csv( '2022-10-17_IDMS_table_descriptions.csv' )

In [88]:
merged_table_info

,table_names,table_vers,created_dates,updated_dates,descriptions,comments
522,DB-STATISTICS,1,1981-08-25,2010-07-16,NaN,NaN
525,DB_STATISTICS,1,1981-08-25,2010-07-16,WORK RECORD FOR DBMG06 MAP,NaN
532,DC-AID-CONDITION-NAMES,1,1981-08-25,2010-07-16,NaN,NaN
1118,MRB-,1,1981-08-25,2010-07-16,NaN,"THIS RECORD IS REQUIRED FOR MAPPING FUNCTIONS,..."
2020,SSCTRL,1,1981-08-25,1994-10-22,NaN,NaN
...,...,...,...,...,...,...
1724,SDDLD560-MAPWORK,4,2020-09-15,2020-09-16,SDDLD560 MAPWORK,NaN
1721,SDDLD560-MAPDAVE,1,2020-09-24,2020-09-24,SDDLD560 MAPWORK,NaN
69,ADPTS601-REC,1,2021-02-19,NaT,UNCLOSED LOANS,NaN
60,ADPTSU01-REC,1,2021-02-19,NaT,GRANT,NaN


# 22's 

In [89]:
text_clumps[22][:20]

['                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ',
 '                      EXT PICT  ZZ9',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ',
 '                      EXT PICT  ZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9']

In [90]:
text_clumps[26][:20]

['                          REDEFINES               W019-RQST-AREA-MFH',
 '                          OCCURS            4',
 "                          VALUE                   'A'",
 "                          VALUE                   'U'",
 '                          REDEFINES               W019-RQST-AREA-MFH',
 '                          OCCURS            4',
 "                          VALUE                   'A'",
 "                          VALUE                   'U'",
 '                          VALUE                   0',
 '                          VALUE                   0',
 '                          REDEFINES               W138-CTY-OFC-MAILG',
 "                          VALUE                   '0'",
 "                          VALUE                   '1'",
 '                          OCCURS           65',
 '                          OCCURS           65',
 '                          OCCURS           12',
 '                          OCCURS           12',
 '                   

In [91]:
text_clumps[27][:20]

['                           100   COMMENT DATA',
 '                           100   OPERATOR',
 '                           200   NUMBER']